# Comparison between different users

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import peakutils

In [3]:
def estimate_period(df, threshold):
    indexes = peakutils.indexes(df['x'].values, thres=threshold, min_dist=10)
    indexes = pd.Series(indexes)
    #peaks = df.iloc[indexes]
    if indexes.index.size != 0:
        diff = indexes.drop(0) - indexes.shift(1).drop(0)
        return diff.mean()
    return float('nan')
    #plt.close()
    #plt.plot(df['x'])
    #plt.plot(peaks['x'], 'r+')

Users who have data on walking: ['/102', '/103', '/107', '/110', '/116', '/118', '/121', '/122', '/125', '/131', '/133', '/140', '/143', '/144', '/148', '/149', '/153', '/159', '/161', '/166', '/171', '/174', '/179', '/181', '/182', '/184', '/188', '/189', '/192']

**Make sure your data file is in the right location on your local machine.**


In [5]:
walking116 = pd.read_csv('../../data/116/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)
walking125 = pd.read_csv('../../data/125/13_treadmill_3mph_0%.csv', names=["tick", "timestamp",
                                             "activity", "x", "y",
                                             "z", "user"], index_col=False)

## Cleaning data
I trimmed the beggining and end of each timeseries due to bogus values.  Points were chosen visually.

In [6]:
walking116 = walking116.iloc[2800:21000]
walking125 = walking125.iloc[2500:20000]
walking116 = walking116.reset_index()
walking125 = walking125.reset_index()

Initialize variables for the search.  Windows will be 1 second long.

In [7]:
window_size = 500
period_upper_116 = []
period_lower_116 = []
period_upper_125 = []
period_lower_125 = []
threshold = .8

In [8]:
for i in np.arange(0, walking116.index.size, window_size):
    period_upper_116.append(estimate_period(walking116.iloc[i:i+window_size], threshold))
    period_lower_116.append(estimate_period(walking116.iloc[i:i+window_size] * -1, threshold))
for i in np.arange(0, walking125.index.size, window_size):
    period_upper_125.append(estimate_period(walking125.iloc[i:i+window_size], threshold))
    period_lower_125.append(estimate_period(walking125.iloc[i:i+window_size] * -1, threshold))

In [9]:
upper = pd.Series(period_upper_116)
lower = pd.Series(period_lower_116)
lower
#print('Upper\n',upper.describe())
#print('Lower\n',lower.describe())

0     104.000000
1     206.000000
2     102.000000
3     102.750000
4     102.666667
5     153.000000
6     102.000000
7     102.000000
8     135.000000
9     101.750000
10    101.750000
11    102.000000
12    101.500000
13    101.750000
14    135.333333
15    101.666667
16    102.000000
17    102.000000
18    102.000000
19    102.000000
20    136.666667
21    102.250000
22    102.250000
23    101.750000
24    101.750000
25    101.000000
26    102.000000
27    135.666667
28    102.666667
29    101.666667
30    102.000000
31    101.500000
32    101.750000
33    102.000000
34    102.333333
35    102.500000
36    104.000000
dtype: float64

## Choosing the correct window size
It seems like get better accuracy with a smaller window size but we can go too small, where our peak detection algorithm does not register any peaks.  Intuitively, our window size cannot be smaller that the shaplet we are searching for